In [2]:
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.objects.log.importer.csv import factory as csv_importer

from pm4py.algo.discovery.dfg import factory as dfg_factory

import pandas as pd

import timeit
import os

In [3]:
method = 'pm4py'

# if you use kubernetes, you can map the CPU_Limit & MEMORY_Limit to the arguments.
# CPU_Limit = os.getenv('CPU_LIMIT')
# MEMORY_Limit = os.getenv('MEMORY_LIMIT')
CPU_Limit = '4.0'
MEMORY_Limit = '1024m'

In [4]:
file = '/data/BPI2016_Clicks_NOT_Logged_In.csv'

In [ ]:
experiment1_file = '/data/experiment1.csv'

In [5]:
result_columns = ['Analysis', 'CPU Limit', 'MEMORY Limit', 'Loading Time', 'DFG Calculation Time', 'Total Time']

In [6]:
def getDFG(result, method, CPU_Limit, MEMORY_Limit, file):
    time_start = timeit.default_timer()

    log = pd.read_csv(file, sep=';', encoding='latin-1', usecols=["SessionID", "PAGE_NAME", "TIMESTAMP"])
    log = log.rename(columns={"SessionID": "case:concept:name", "PAGE_NAME": "concept:name", "TIMESTAMP": "time:timestamp"})

    time_log = timeit.default_timer()

    dfg = dfg_factory.apply(log)

    time_stop = timeit.default_timer()
    

    Loading_Time = time_log-time_start
    DFG_Calculation_Time = time_stop-time_log
    Total_Time = time_stop-time_start

    new_row = pd.DataFrame([[method, CPU_Limit, MEMORY_Limit, Loading_Time, DFG_Calculation_Time, Total_Time]]
                           , columns=result_columns)

    result = pd.concat([result, new_row])
    
    return result

In [7]:
if os.path.isfile(experiment1_file):
    result = pd.read_csv(experiment1_file)
else:
    result = pd.DataFrame(columns=result_columns)

In [9]:
result = getDFG(result, method, CPU_Limit, MEMORY_Limit, file)

In [10]:
result = result.sort_values(by=result_columns)
result.to_csv(experiment1_file, index = False)